In [ ]:
from Data_manager.BookData.BookDataReader import BookDataReader

dataset_object = BookDataReader()
dataset = dataset_object.load_data()

URM_all = dataset.get_URM_from_name("URM_all")
ICM_all = dataset.get_ICM_from_name("ICM_all")

In [ ]:
from Base.DataIO import DataIO
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_user_wise

OUTPUT_path = "Result_experiments/"

data_loader = DataIO(folder_path=OUTPUT_path)

URM_train, URM_test = split_train_in_two_percentage_user_wise(URM_all, train_percentage=0.8)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
dict = {}
graph_dict = {}

In [ ]:
def load_recommender(recommender_class, URM, ICM=None, metadata_args=""):
    metadata_path = recommender_class.RECOMMENDER_NAME + metadata_args + "_metadata.zip"
    metadata = data_loader.load_data(metadata_path)
    best_parameters = metadata["hyperparameters_best"]

    if ICM is None:
        recommender = recommender_class(URM)
    else:
        recommender = recommender_class(URM, ICM)

    recommender.fit(**best_parameters)

    return recommender

def set_recommender(dict, recommender, key=None):
    if key is None:
        key = recommender.RECOMMENDER_NAME

    dict[key] = {}
    dict[key]["model"] = recommender

In [ ]:
from Base.Recommender_utils import check_matrix
from Base.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender

import numpy as np


class CalibrationRecommender(BaseItemSimilarityMatrixRecommender):

    RECOMMENDER_NAME = "CalibrationRecommender"

    def __init__(self, URM_train, Recommender):
        super(CalibrationRecommender, self).__init__(URM_train)

        self.URM_train = check_matrix(URM_train.copy(), "csr")

        self.Recommender = Recommender
    
    def fit(self):

        item_weights = self.Recommender._compute_item_score(list(range(self.URM_train.shape[0])))

        self.item_vmax_mean = []
        for i in range(self.n_items):
            item_scores = item_weights[:, i][item_weights[:, i] > 0]

            if len(item_scores) == 0:
                item_scores = [0]

            self.item_vmax_mean.append({"max": np.max(item_scores), "mean": np.mean(item_scores)})

    def _compute_item_score(self, user_id_array, items_to_compute=None):

        item_weights = self.Recommender._compute_item_score(user_id_array, items_to_compute)

        for index, user_id in enumerate(user_id_array):
            user_scores = item_weights[index, :]

            for i, score in enumerate(user_scores):
                vmax = self.item_vmax_mean[i]["max"]
                mean = self.item_vmax_mean[i]["mean"]
                
                if mean != 0:
                    r = user_scores[i]
                    user_scores[i] = ((r / mean) * 0.5) if r <= mean else (0.5 + (r - mean) / (vmax - mean) * 0.5) if r <= vmax else 1

            item_weights[index, :] = user_scores

        return item_weights


### ItemKNNCFRecommender

In [ ]:
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommender = load_recommender(ItemKNNCFRecommender, URM_train, metadata_args="_cosine")

set_recommender(dict, recommender)

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

In [ ]:
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommender = ItemKNNCFRecommender(URM_train)
recommender.fit(topK=337, shrink=722, similarity="asymmetric", asymmetric_alpha=0.11770705488555902, feature_weighting="TF-IDF")

set_recommender(dict, recommender)

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### P3alphaRecommender

In [ ]:
from GraphBased.P3alphaRecommender import P3alphaRecommender

recommender = load_recommender(P3alphaRecommender, URM_train)

set_recommender(dict, recommender)

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

In [ ]:
from GraphBased.P3alphaRecommender import P3alphaRecommender

recommender = P3alphaRecommender(URM_train)
recommender.fit(topK=400, alpha=0.4, normalize_similarity=False, implicit=True)

set_recommender(dict, recommender)

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### RP3betaRecommender

In [ ]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

recommender = RP3betaRecommender(URM_train)
recommender.fit(topK=906 , alpha=0.5, beta=0.08, normalize_similarity=False, implicit=True)

set_recommender(dict, recommender)

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

In [ ]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

recommender = load_recommender(RP3betaRecommender, URM_train)

set_recommender(dict, recommender)

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### ItemKNNCBFRecommender

In [ ]:
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

recommender = load_recommender(ItemKNNCBFRecommender, URM_train, ICM=ICM_all, metadata_args="_ICM_all_cosine")

set_recommender(dict, recommender)

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

In [ ]:
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

recommender = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender.fit(topK=142, shrink=60, similarity="jaccard", normalize=False)

set_recommender(dict, recommender)

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### ItemKNNCFRecommender_ItemKNNCBFRecommender

In [ ]:
from Hybrid.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender

RECOMMENDER_1 = dict[ItemKNNCFRecommender.RECOMMENDER_NAME]["model"]
RECOMMENDER_2 = dict[ItemKNNCBFRecommender.RECOMMENDER_NAME]["model"]

recommender = ItemKNNScoresHybridRecommender(URM_train, RECOMMENDER_1, RECOMMENDER_2)
recommender.fit(alpha=0.4)

set_recommender(dict, recommender, key="ItemKNNCFRecommender_ItemKNNCBFRecommender")
set_recommender(graph_dict, recommender, key="ItemKNNCFRecommender_ItemKNNCBFRecommender")

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### P3alphaRecommender_ItemKNNCBFRecommender

In [ ]:
from Hybrid.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender

RECOMMENDER_1 = dict[P3alphaRecommender.RECOMMENDER_NAME]["model"]
RECOMMENDER_2 = dict[ItemKNNCBFRecommender.RECOMMENDER_NAME]["model"]

recommender = ItemKNNScoresHybridRecommender(URM_train, RECOMMENDER_1, RECOMMENDER_2)
recommender.fit(alpha=0.4)

set_recommender(dict, recommender, key="P3alphaRecommender_ItemKNNCBFRecommender")
set_recommender(graph_dict, recommender, key="P3alphaRecommender_ItemKNNCBFRecommender")

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### RP3betaRecommender_ItemKNNCBFRecommender

In [ ]:
from Hybrid.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender

RECOMMENDER_1 = dict[RP3betaRecommender.RECOMMENDER_NAME]["model"]
RECOMMENDER_2 = dict[ItemKNNCBFRecommender.RECOMMENDER_NAME]["model"]

recommender = ItemKNNScoresHybridRecommender(URM_train, RECOMMENDER_1, RECOMMENDER_2)
recommender.fit(alpha=0.6)

set_recommender(dict, recommender, key="RP3betaRecommender_ItemKNNCBFRecommender")
set_recommender(graph_dict, recommender, key="RP3betaRecommender_ItemKNNCBFRecommender")

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### P3alphaRecommender_ItemKNNCFRecommender_ItemKNNCBFRecommender

In [ ]:
from Hybrid.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender

RECOMMENDER_1 = dict[P3alphaRecommender.RECOMMENDER_NAME]["model"]
RECOMMENDER_2 = dict[ItemKNNCFRecommender.RECOMMENDER_NAME]["model"]
RECOMMENDER_3 = dict[ItemKNNCBFRecommender.RECOMMENDER_NAME]["model"]

temporary = ItemKNNScoresHybridRecommender(URM_train, RECOMMENDER_1, RECOMMENDER_2)
temporary.fit(alpha=0.9)

recommender = ItemKNNScoresHybridRecommender(URM_train, temporary, RECOMMENDER_3)
recommender.fit(alpha=0.4)

set_recommender(dict, recommender, key="P3alphaRecommender_ItemKNNCFRecommender_ItemKNNCBFRecommender")
set_recommender(graph_dict, recommender, key="P3alphaRecommender_ItemKNNCFRecommender_ItemKNNCBFRecommender")

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### RP3betaRecommender_ItemKNNCFRecommender_ItemKNNCBFRecommender

In [ ]:
from Hybrid.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender

RECOMMENDER_1 = dict[RP3betaRecommender.RECOMMENDER_NAME]["model"]
RECOMMENDER_2 = dict[ItemKNNCFRecommender.RECOMMENDER_NAME]["model"]
RECOMMENDER_3 = dict[ItemKNNCBFRecommender.RECOMMENDER_NAME]["model"]

temporary = ItemKNNScoresHybridRecommender(URM_train, RECOMMENDER_1, RECOMMENDER_2)
temporary.fit(alpha=0.9)

recommender = ItemKNNScoresHybridRecommender(URM_train, temporary, RECOMMENDER_3)
recommender.fit(alpha=0.4)

set_recommender(dict, recommender, key="RP3betaRecommender_ItemKNNCFRecommender_ItemKNNCBFRecommender")
set_recommender(graph_dict, recommender, key="RP3betaRecommender_ItemKNNCFRecommender_ItemKNNCBFRecommender")

results, results_string = evaluator_test.evaluateRecommender(recommender)
print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_string))

### ColdUsersTopPopRecommender

In [ ]:
from Hybrid.ColdUsersTopPopRecommender import ColdUsersTopPopRecommender

for key in graph_dict.keys():
    recommender = graph_dict[key]["model"]

    temporary = ColdUsersTopPopRecommender(URM_train, recommender)
    temporary.fit()

    set_recommender(graph_dict, temporary, key=key)

### UserWiseRecommender

In [ ]:
import scipy.sparse as sps
import numpy as np

URM_train = sps.csr_matrix(URM_train)
profile_length = np.ediff1d(URM_train.indptr)

In [ ]:
n_groups = 5
block_size = int(len(profile_length) / n_groups)

In [ ]:
sorted_users = np.argsort(profile_length)

In [ ]:
cutoff = 10

for key in graph_dict.keys():
    graph_dict[key]["MAP_per_group"] = []

for group_id in range(0, n_groups):
    start_pos = group_id * block_size
    end_pos = min((group_id + 1) * block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, {}, average p.len {:.2f}, min {}, max {}".format(
        group_id, len(users_in_group), users_in_group_p_len.mean(), users_in_group_p_len.min(), users_in_group_p_len.max()))
    
    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    users_not_in_group = sorted_users[users_not_in_group_flag]
    
    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)

    for key in graph_dict.keys():
        recommender = graph_dict[key]
        results, _ = evaluator_test.evaluateRecommender(recommender["model"])

        recommender["MAP_per_group"].append(results[cutoff]["MAP"])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(15, 10))

for key in graph_dict.keys():
    recommender = graph_dict[key]
    plt.plot(recommender["MAP_per_group"], label=key)

plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()